In [ ]:
from pyspark.sql.functions import col, expr, sum, month, count, year, dayofmonth, hour, minute, dayofweek, unix_timestamp, avg, stddev


In [ ]:
# Sparkを使用して、指定されたParquetファイルを読み込みます。この操作により、dfというSpark DataFrameが作成されます。
df = spark.read.parquet("Files/Bronze/2019/1/green_tripdata_2019-01.parquet")
# df now is a Spark DataFrame containing parquet data from "Files/Bronze/2019/1/green_tripdata_2019-01.parquet".
display(df)

In [ ]:
#データフレームのスキーマを表示します。
df.printSchema()

In [ ]:
#レコード数の確認
record_count = df.count()
print(f"Total number of records: {record_count}")

In [ ]:
# 各列の基本統計情報を表示。
display(df.describe())
# count: 各列に含まれる非NULLエントリの数。
# mean: 各列の平均値。
# stddev: 各列の標準偏差。
# min: 各列の最小値。
# max: 各列の最大値。

In [ ]:
#データフレームから分析に不要な列を取り除き、データ量を減らします。
df_drop = df.drop("VendorID","store_and_fwd_flag","RateCodeID","extra", "mta_tax", "tolls_amount", "ehail_fee","improvement_surcharge","fare_amount","tip_amount","Trip_type","congestion_surcharge")

In [ ]:
#データの確認。列が削除されていることを確認します。
display(df_drop)

In [ ]:
#レコード数の確認
record_count = df_drop.count()
print(f"Total number of records: {record_count}")

In [ ]:
# dropnaメソッドを使用して、df_dropデータフレームからNULL値（欠損値）が含まれている行を削除します。
df_drop_na = df_drop.dropna()

In [ ]:
#レコード数の確認
record_count = df_drop_na.count()
print(f"Total number of records: {record_count}")

In [ ]:
# 欠損値の数をカウント

# 各列の欠損値（NULL値）の数をカウント
null_counts = df_drop_na.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_drop_na.columns])
# df_drop_na.columnsでデータフレームのすべての列名を取得します。
# リスト内包表記を使用して、各列に対して以下の処理を行います：
# 1. col(c).isNull()で、列cの値がNULLであるかどうかをチェックします。
# 2. .cast("int")で、ブール値を整数にキャストします（NULLであれば1、そうでなければ0）。
# 3. sum()で、列内のすべてのNULL値の合計を計算します。
# 4. alias(c)で、結果の列名を元の列名に設定します。
# 欠損値の数を表示
display(null_counts)

In [ ]:
# df_drop_naの基本統計情報を表示
# count数がすべて同じであることがわかります。
display(df_drop_na.describe())

In [ ]:
#lpep_pickup_datetime/lpep_dropoff_datetimeに該当期間ではないデータが入っていることがあるので適正な期間に修正
#例えば、2019年1月のデータに2019年2月の期間のデータが入っていることもあります。
df_filtered = df_drop_na
# 月ごとの分布を計算
# df_filteredデータフレームを以下のキーでグループ化します：
# 1. lpep_pickup_datetime列の月を抽出して"pickup_month"としてエイリアスを設定
# 2. lpep_dropoff_datetime列の月を抽出して"dropoff_month"としてエイリアスを設定
month_distribution = df_filtered.groupBy(
    month(col("lpep_pickup_datetime")).alias("pickup_month"),
    month(col("lpep_dropoff_datetime")).alias("dropoff_month")
).agg(
    # 各グループの行数をカウントし、その結果に"count"としてエイリアスを設定
    count("*").alias("count")
)

In [ ]:
#display関数を使用して、計算されたmonth_distributionデータフレームを表示します。これにより、各ピックアップ月とドロップオフ月の組み合わせごとのレコード数を視覚的に確認できます。どちらも1月のものが該当のものです。
display(month_distribution)

In [ ]:
#filterメソッドを使用して、df_filteredデータフレームから特定の条件を満たす行をフィルタリングします。具体的な条件は以下の通りです：
#lpep_pickup_datetime列の月が1月である。
#lpep_dropoff_datetime列の月が1月である。
df_filtered_month = df_filtered.filter((month(col("lpep_pickup_datetime")) == 1) & (month(col("lpep_dropoff_datetime")) == 1))
# 1月のデータで月ごとの分布を計算
month_distribution = df_filtered_month.groupBy(
    # lpep_pickup_datetime列の月を抽出して"pickup_month"としてエイリアスを設定
    month(col("lpep_pickup_datetime")).alias("pickup_month"),
    # lpep_dropoff_datetime列の月を抽出して"dropoff_month"としてエイリアスを設定
    month(col("lpep_dropoff_datetime")).alias("dropoff_month")
).agg(
    # 各グループの行数をカウントし、その結果に"count"としてエイリアスを設定
    count("*").alias("count")
)

In [ ]:
#display関数を使用して、計算されたmonth_distributionデータフレームを表示します。これにより、各ピックアップ月とドロップオフ月の組み合わせごとのレコード数を視覚的に確認できます。どちらも1月のものが該当のものです。
display(month_distribution)

In [ ]:
#レコード数の確認
record_count = df_filtered_month.count()
print(f"Total number of records: {record_count}")

In [ ]:
# lpep_pickup_datetimeを年、月、日、時間、分、曜日に分ける
# dayofweek関数は曜日を1（=日曜日）から7（=土曜日）で返します。
df_filtered_month = df_filtered_month.withColumn("pickup_year", year(col("lpep_pickup_datetime"))) \
        .withColumn("pickup_month", month(col("lpep_pickup_datetime"))) \
        .withColumn("pickup_day", dayofmonth(col("lpep_pickup_datetime"))) \
        .withColumn("pickup_hour", hour(col("lpep_pickup_datetime"))) \
        .withColumn("pickup_minute", minute(col("lpep_pickup_datetime"))) \
        .withColumn("pickup_weekday", dayofweek(col("lpep_pickup_datetime")))


In [ ]:
# lpep_dropoff_datetimeとの差分を計算し、trip_durationとして追加
df_filtered_month = df_filtered_month.withColumn("trip_duration", 
                (unix_timestamp(col("lpep_dropoff_datetime")) - unix_timestamp(col("lpep_pickup_datetime"))) / 60)

# lpep_pickup_datetimeとlpep_dropoff_datetimeを削除
df_filtered_month = df_filtered_month.drop("lpep_dropoff_datetime")

In [ ]:
#レコード数の確認
record_count = df_filtered_month.count()
print(f"Total number of records: {record_count}")

In [ ]:
display(df_filtered_month.describe())


In [ ]:
df_filtered_month.printSchema()


In [ ]:
# フィルタリングされた1月のデータの型を変換
df_filtered_month = df_filtered_month.selectExpr(
    "cast(lpep_pickup_datetime as timestamp) as pickup_datetime",  # lpep_pickup_datetime列をtimestamp型に変換
    "cast(PULocationID as int) as pickup_LocationID",
    "cast(DOLocationID as int) as dropoff_LocationID",
    "cast(payment_type as int) as payment_type",
    "cast(passenger_count as int) as passenger_count",
    "cast(trip_distance as double) as trip_distance",
    "cast(total_amount as double) as total_amount",
    "cast(pickup_year as int) as pickup_year",
    "cast(pickup_month as int) as pickup_month",
    "cast(pickup_day as int) as pickup_day",
    "cast(pickup_hour as int) as pickup_hour",
    "cast(pickup_minute as int) as pickup_minute",
    "cast(pickup_weekday as int) as pickup_weekday",
    "cast(trip_duration as double) as trip_duration"
)

In [ ]:
# データの値を制限
df_filtered_numbers = df_filtered_month.filter(
    (col("pickup_LocationID").between(1, 265)) & 
    (col("dropoff_LocationID").between(1, 265)) & 
    (col("payment_type").between(1, 5)) & 
    (col("passenger_count").between(1, 6)) & 
    (col("trip_distance") > 0) & 
    (col("total_amount") > 0) & 
    (col("pickup_year").between(2019, 2021)) &  # pickup_yearに対するフィルタリング条件
    (col("pickup_month").between(1, 12)) &  # pickup_monthに対するフィルタリング条件
    (col("pickup_day").between(1, 31)) &  # pickup_dayに対するフィルタリング条件
    (col("pickup_hour").between(0, 23)) &  # pickup_hourに対するフィルタリング条件
    (col("pickup_minute").between(0, 59)) &  # pickup_minuteに対するフィルタリング条件
    (col("trip_duration") > 0)  # trip_durationに対するフィルタリング条件
)

In [ ]:
# df_filtered_numbersの基本統計情報を表示。trip_distance, total_amount, trip_durationマイナスが無いことを確認
display(df_filtered_numbers.describe())

In [ ]:
df_filtered_numbers.printSchema()

In [ ]:
#レコード数の確認
record_count = df_filtered_numbers.count()
print(f"Total number of records: {record_count}")

In [ ]:
# passenger_countごとにグループ化し、その出現回数をカウント
value_counts_df = df_filtered_numbers.groupBy("passenger_count").count().orderBy(col("passenger_count").asc())
display(value_counts_df)

In [ ]:
## zscoreを計算
# 平均と標準偏差の計算
stats = df_filtered_numbers.select(avg(col('trip_duration')).alias('mean'), stddev(col('trip_duration')).alias('stddev')).collect()

mean_trip_duration = stats[0]['mean']
stddev_trip_duration = stats[0]['stddev']

# 異常値の除去（Zスコアがまたは-2を超える値を除去）
df_filtered_numbers = df_filtered_numbers.filter((col('trip_duration') - mean_trip_duration) / stddev_trip_duration <= 2) \
                                     .filter((col('trip_duration') - mean_trip_duration) / stddev_trip_duration >= -2)


In [ ]:
# IQRを計算して外れ値の範囲を決定
# trip_distance列の第1四分位数（Q1）と第3四分位数（Q3）を近似的に計算

quantiles = df_filtered_numbers.approxQuantile("trip_distance", [0.25, 0.75], 0.01)
Q1, Q3 = quantiles[0], quantiles[1]

# 四分位範囲（IQR）を計算
IQR = Q3 - Q1

# 外れ値を検出するための下限と上限を計算
# 外れ値を検出するための下限 (lower_bound) を Q1 - 1.5 * IQR で計算します。
lower_bound = Q1 - 1.5 * IQR
# 外れ値を検出するための上限 (upper_bound) を Q3 + 1.5 * IQR で計算します。
upper_bound = Q3 + 1.5 * IQR

# 外れ値をフィルタリング
# trip_distance列の値が下限以上かつ上限以下の行をフィルタリング
df_iqr= df_filtered_numbers.filter(
(col("trip_distance") >= lower_bound) & 
(col("trip_distance") <= upper_bound)
)

In [ ]:
df_trimming = df_iqr

In [ ]:
# 上位0.5%と下位0.5%の境界値を計算
# total_amount列の下位0.5%と上位0.5%の境界値を近似的に計算
bounds = df_trimming.approxQuantile("total_amount", [0.005, 0.995], 0.0)

# 境界値を取得
lower_bound = bounds[0]  # 下位0.5%の境界値
upper_bound = bounds[1]  # 上位0.5%の境界値

# 境界値を超える外れ値をフィルタリング
# total_amount列の値が下限以上かつ上限以下の行をフィルタリング
df_trimming = df_trimming.filter((col("total_amount") >= lower_bound) & (col("total_amount") <= upper_bound))

# 境界値を表示

print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")


In [ ]:
display(df_trimming.describe())


In [ ]:
#レコード数の確認
record_count = df_trimming.count()
print(f"Total number of records: {record_count}")